# Simulator

In [1]:
import pandas as pd

import payment_simulator as ps
from payment_simulator.anomaly import AnomalyGenerator
from payment_simulator.networks import SimplePaymentNetwork
from payment_simulator.utils import random_payment_timing, random_payment_value

In [2]:
TOTAL_BANKS = 20
INITIAL_BANKS = 3
INCREMENT_BANKS = 5
ALPHA = 0.00001
AVG_PAYMENTS = 35
ALLOW_SELF_LOOP = False

In [3]:
sim_periods = list(range(15))

sim_params = {
    "open_time": "06:30:00",
    "close_time": "18:30:00",
    "value_fn": random_payment_value,
    "timing_fn": random_payment_timing,
}

payment_network = SimplePaymentNetwork(
    total_banks=TOTAL_BANKS,
    avg_payments=AVG_PAYMENTS,
    alpha=ALPHA,
    allow_self_loop=ALLOW_SELF_LOOP
)

anomaly_generator = AnomalyGenerator(
    anomaly_start=5,
    anomaly_end=12,
    prob_start=0.8,
    prob_end=1,
    lambda_start=0.5,
    lambda_end=2.5,
    rate=0.5,
)

In [4]:
normal_transactions = ps.TransactionSim(sim_id=1, network=payment_network, **sim_params)
normal_transactions.run(sim_periods)

payments1 = normal_transactions.get_payments_df()

pd.concat([payments1.head(3), payments1.tail(3)])

,Period,Time,Sender,Receiver,Count,Value
0,0,09:57:14,0,5,1,0.681788
1,0,08:18:40,0,5,1,2.203179
2,0,08:16:58,0,5,1,0.139019
10497,14,10:45:12,18,3,1,4.353783
10498,14,17:12:32,18,6,1,0.088409
10499,14,15:46:58,18,8,1,0.169963


In [5]:
anomaly_transactions = ps.AnomalyTransactionSim(sim_id=2, network=payment_network, anomaly=anomaly_generator, **sim_params)
anomaly_transactions.run(sim_periods)

payments2 = anomaly_transactions.get_payments_df()

pd.concat([payments2.head(3), payments2.tail(3)])

,Period,Time,Sender,Receiver,Count,Value
0,0,10:15:20,0,9,1,0.379964
1,0,06:55:46,0,9,1,1.385770
2,0,15:16:33,0,9,1,1.485390
10497,14,11:34:29,18,14,1,0.943585
10498,14,11:10:36,19,14,1,1.305875
10499,14,10:09:05,19,3,1,1.883888


In [6]:
print(f"Total Value of Normal RTGS  : {payments1['Value'].sum():.3f} from {payments1.shape[0]} transactions")
print(f"Total Value of Anomaly RTGS : {payments2['Value'].sum():.3f} from {payments1.shape[0]} transactions")

Total Value of Normal RTGS  : 17277.613 from 10500 transactions
Total Value of Anomaly RTGS : 26468.346 from 10500 transactions


In [7]:
correct = 0
test_len = 5

for _ in range(test_len):
    normal_transactions.run(sim_periods)
    anomaly_transactions.run(sim_periods)

    normal_df = normal_transactions.get_payments_df()
    normal_df = normal_df[(normal_df["Period"] <=12) & (normal_df["Period"] >= 5)]

    anomaly_df = anomaly_transactions.get_payments_df()
    anomaly_df = anomaly_df[(anomaly_df["Period"] <=12) & (normal_df["Period"] >= 5)]

    x1 = normal_df["Value"].sum()
    x2 = anomaly_df["Value"].sum()

    if x2 > x1:
        correct += 1

print(f"Success rate: {correct / test_len * 100:.2f}%")

Success rate: 100.00%
